In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Tried to do some EDA and created some simple models on the dataset. Please check and provide your feedback.

**Work Still In Progress

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot, download_plotlyjs
init_notebook_mode(connected = True)

In [ ]:
# Function defined to check medata of a dataframe
def master_dataframe(dataframe):
    df_metadata = pd.DataFrame({'Datatype': dataframe.dtypes,
                                "Null Values": dataframe.isna().sum(),  
                                "Null %": round(dataframe.isna().sum()/len(dataframe)*100, 2),
                                "No: Of Unique Values": dataframe.nunique()})
    
    df_describe = dataframe.describe(include='all').T
    
    df_metadata = df_metadata.join(df_describe)  

    return df_metadata

In [ ]:
# Importing data
raw_data = pd.read_csv('../input/health-care-data-set-on-heart-attack-possibility/heart.csv')

In [ ]:
# Let's check the metadata of the dataset.
master_dataframe(raw_data)

Some observations about the data from the above table :
1. There are no null values.
2. There are only integer\float values hence no need of encoding required for categorical value like sex. 
   Assuming 0 is for female and 1 is for male.
3. Values of the features are in different scales hence feature scaling will be necessary.

# Exploratory Data Analysis & Data Visualisation

Study shows a fasting Blood Glucose of above 100 increases risk of heart disease by 300%. People with a fasting blood sugar level of 100-125 mg/dl had an adjusted nearly 300% increase higher risk of having coronary heart disease than people with a level below 79 mg/dl.

In [ ]:
# Lets check the heart attack possiblilty based on gender, exercise induced angina and fasting blood sugar.

grp_by_sex = raw_data.groupby(by=['sex'])['target'].sum().reset_index()
grp_by_sex['sex'] = grp_by_sex['sex'].map({0:'Female', 1:'Male'})

grp_by_fbs = raw_data.groupby(by=['fbs'])['target'].sum().reset_index()
grp_by_fbs['fbs'] = grp_by_fbs['fbs'].map({1:'>120mg/dl', 0:'<120mg/dl'}) # 1 : fbs <120mg/dl , 0 : fbs >120mg/dl

grp_by_ex = raw_data.groupby(by=['exang'])['target'].sum().reset_index()
grp_by_ex['exang'] = grp_by_ex['exang'].map({0:'Yes', 1:'No'}) # 1 : fbs <120mg/dl , 0 : fbs >120mg/dl

fig = make_subplots(rows = 1, cols = 3, subplot_titles = ("By Gender", "By FBS", "By Exercise Induced Angina"))

fig.add_trace(go.Pie(values = grp_by_sex['target'], 
                     labels = grp_by_sex['sex'],
                     domain = dict(x=[0, 0.30]), 
                     name = 'Gender', 
                     showlegend = False, 
                     marker = dict(line = {'color': 'black', 'width': 0.5})))

fig.add_trace(go.Pie(values = grp_by_fbs['target'], 
                     labels = grp_by_fbs['fbs'],
                     domain = dict(x=[0.33, 0.60]),
                     name = 'FBS',
                     showlegend = False, 
                     marker = dict(line = {'color': 'black', 'width': 0.5})))


fig.add_trace(go.Pie(values = grp_by_ex['target'], 
                     labels = grp_by_ex['exang'], 
                     domain = dict(x=[0.66, 1]),
                     name = 'Exercise Induced Angina',
                     showlegend = False,
                     marker = dict(line = {'color': 'black', 'width': 0.5})))


fig.update_layout(title = dict(text = 'Heart Attacks Experienced',
                                x = 0.5,
                                y = 0.98,
                                font = {'size': 20}),
                  hovermode = 'closest')


iplot(fig)

# We can see males are at more risk of a heart attack than females.

The above charts show that : 
1. Males are at more risk of heart attacks as compared to females.
2. People with high fasting blood sugar levels have experienced heart attacks.
3. People with exercise induced angina have had heart attacks. 

In [ ]:
# Lets check the heart attack possiblilty based on gender, exercise induced angina and fasting blood sugar.
grp_by_cp = raw_data.groupby(by=['cp'])['target'].sum().reset_index()

grp_by_thal = raw_data.groupby(by=['thal'])['target'].sum().reset_index()
grp_by_thal['thal'] = grp_by_thal['thal'].map({0:'Normal', 1:'Fixed Defect', 2: 'Reversible Defect'}) 

grp_by_recg = raw_data.groupby(by=['restecg'])['target'].sum().reset_index()
grp_by_recg['restecg'] = grp_by_recg['restecg'].map({0:'Normal', 1:'ST-T Wave Abnormality', 2: 'Left Ventricular Hypertrophy'})

grp_by_st = raw_data.groupby(by=['slope'])['target'].sum().reset_index()

grp_by_ca = raw_data.groupby(by=['ca'])['target'].sum().reset_index()

fig = make_subplots(rows = 5, 
                    cols = 1, 
                    subplot_titles = ("By Chest Pain", "By Rest ECG", "By Slope ST Segment", "By CA", "By THAL"))

fig.add_trace(go.Bar(y = grp_by_cp['target'], 
                     x = grp_by_cp['cp'],
                     name = 'Chest Pain',
                     text = grp_by_cp['target'],
                     textposition = 'inside',
                     showlegend = False, 
                     marker = dict(line = {'color': 'black', 'width': 0.5})), 
              row = 1, col = 1)

fig.add_trace(go.Bar(y = grp_by_recg['target'], 
                     x = grp_by_recg['restecg'],
                     text = grp_by_recg['target'],
                     textposition = 'inside',
                     showlegend = False, 
                     marker = dict(line = {'color': 'black', 'width': 0.5})), 
              row = 2, col = 1)


fig.add_trace(go.Bar(y = grp_by_st['target'], 
                     x = grp_by_st['slope'], 
                     text = grp_by_st['target'],
                     textposition = 'inside',
                     showlegend = False,
                     marker = dict(line = {'color': 'black', 'width': 0.5})), 
              row = 3, col = 1)

fig.add_trace(go.Bar(y = grp_by_ca['target'], 
                     x = grp_by_ca['ca'], 
                     text = grp_by_ca['target'],
                     textposition = 'inside',
                     showlegend = False,
                     marker = dict(line = {'color': 'black', 'width': 0.5})), 
              row = 4, col = 1)

fig.add_trace(go.Bar(y = grp_by_thal['target'], 
                     x = grp_by_thal['thal'], 
                     text = grp_by_thal['target'],
                     textposition = 'inside',
                     showlegend = False,
                     marker = dict(line = {'color': 'black', 'width': 0.5})), 
              row = 5, col = 1)


fig.update_layout(title = dict(text = 'Heart Attacks Experienced',
                                x = 0.5,
                                y = 0.98,
                                font = {'size': 20}),
                  hovermode = 'closest', 
                  height = 1500, 
                  width = 500)

fig.update_xaxes(title_text = 'Chest Pain', 
                 row = 1, 
                 col = 1)

#fig.update_xaxes(title_text = 'Rest ECG', 
                 #row = 2, 
                # col = 1)

fig.update_xaxes(title_text = 'Slope', 
                 row = 3, 
                 col = 1)

fig.update_xaxes(title_text = 'CA', 
                 row = 4, 
                 col = 1)

fig.update_xaxes(title_text = 'THAL', 
                 row = 5, 
                 col = 1)

fig.update_yaxes(title_text = 'Heart Attacks', 
                 range = [0, 100],
                 row = 1, 
                 col = 1)

fig.update_yaxes(title_text = 'Heart Attacks',
                 range = [0, 100],
                 row = 2, 
                 col = 1)

fig.update_yaxes(title_text = 'Heart Attacks', 
                 range = [0, 120],
                 row = 3, 
                 col = 1)

fig.update_yaxes(title_text = 'Heart Attacks', 
                 range = [0, 150],
                 row = 4, 
                 col = 1)

fig.update_yaxes(title_text = 'Heart Attacks',
                 range = [0, 150],
                 row = 5, 
                 col = 1)

iplot(fig)



We got great insights from the above charts for all the categorical features.
Lets check the relationships between continous features now.

In [ ]:
sns.pairplot(raw_data)
plt.show()

In [ ]:
data = [go.Scatter(x = raw_data['chol'], 
                   y = raw_data['trestbps'], 
                   mode = 'markers', 
                   marker = dict(symbol = 'diamond', 
                                 opacity = 0.8,
                                 size = 16))]

layout = go.Layout(title = dict(text = 'Cholestrol Vs Resting Blood Pressure', 
                                x = 0.5, 
                                y = 0.88), 
                   xaxis = dict(title = 'Cholestrol', 
                                titlefont = {'size': 16}), 
                   yaxis = dict(title = 'Resting Blood Pressure', 
                                titlefont = {'size': 16}, 
                                range = [0, 250]))

fig = go.Figure(data = data, 
                layout = layout)

iplot(fig)

In [ ]:
data = [go.Scatter(x = raw_data['chol'], 
                   y = raw_data['thalach'], 
                   mode = 'markers', 
                   marker = dict(symbol = 'diamond', 
                                 color = raw_data['target'],
                                 showscale = True,
                                 opacity = 0.8,
                                 size = 16))]

layout = go.Layout(title = dict(text = 'Cholestrol Vs Maximum Heart Rate Achieved - Heart Attack', 
                                x = 0.5, 
                                y = 0.88), 
                   xaxis = dict(title = 'Cholestrol', 
                                titlefont = {'size': 16}), 
                   yaxis = dict(title = 'Maximum Heart Rate Achieved', 
                                titlefont = {'size': 16}, 
                                range = [0, 250]))

fig = go.Figure(data = data, 
                layout = layout)

iplot(fig)

In [ ]:
# Lets split our Dependent & Independent features.
X = raw_data.iloc[:, :-1].values
y = raw_data.iloc[:, -1].values

In [ ]:
#Lets split out data into training and test set.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)
print('Shape of X_train: ', X_train.shape)
print('Shape of X_test: ', X_test.shape)
print('Shape of y_train: ', y_train.shape)
print('Shape of y_test: ', y_test.shape)

In [ ]:
# Lets scale our data.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_classifier = LogisticRegression()
lr_classifier.fit(X_train, y_train)

y_pred_lr = lr_classifier.predict(X_test)

# Lets measure the accuracy of the model using the confusion matrix.
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred_lr)

corr_pred = cm[0, 0] + cm[1, 1]
total = cm.sum()
corr_pred_per = round(corr_pred/total*100, 2)
print('Percentage of correct predictions: ', corr_pred_per)


# K Nearst Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
knn_classifier.fit(X_train, y_train)

y_pred_knn = knn_classifier.predict(X_test)

# Lets measure the accuracy of the model using the confusion matrix.
cm = confusion_matrix(y_test, y_pred_knn)

corr_pred = cm[0, 0] + cm[1, 1]
total = cm.sum()
corr_pred_per_knn = round(corr_pred/total*100, 2)
print('Percentage of correct predictions: ', corr_pred_per_knn)

# Support Vector Machine

In [ ]:
from sklearn.svm import SVC
svc_classifier = SVC(kernel = 'linear')
svc_classifier.fit(X_train, y_train)

y_pred_svc = svc_classifier.predict(X_test)

# Lets measure the accuracy of the model using the confusion matrix.
cm = confusion_matrix(y_test, y_pred_svc)

corr_pred = cm[0, 0] + cm[1, 1]
total = cm.sum()
corr_pred_per_svc = round(corr_pred/total*100, 2)
print('Percentage of correct predictions: ', corr_pred_per_svc)

# Kernel SVM

In [ ]:
svm_classifier = SVC(kernel = 'rbf')
svm_classifier.fit(X_train, y_train)

y_pred_svm = svm_classifier.predict(X_test)

# Lets measure the accuracy of the model using the confusion matrix.
cm = confusion_matrix(y_test, y_pred_svm)

corr_pred = cm[0, 0] + cm[1, 1]
total = cm.sum()
corr_pred_per_svm = round(corr_pred/total*100, 2)
print('Percentage of correct predictions: ', corr_pred_per_svm)

# Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

y_pred_nb = nb_classifier.predict(X_test)

# Lets measure the accuracy of the model using the confusion matrix.
cm = confusion_matrix(y_test, y_pred_nb)

corr_pred = cm[0, 0] + cm[1, 1]
total = cm.sum()
corr_pred_per_nb = round(corr_pred/total*100, 2)
print('Percentage of correct predictions: ', corr_pred_per_nb)

# Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(criterion = 'entropy')
dt_classifier.fit(X_train, y_train)

y_pred_dt = dt_classifier.predict(X_test)

# Lets measure the accuracy of the model using the confusion matrix.
cm = confusion_matrix(y_test, y_pred_dt)

corr_pred = cm[0, 0] + cm[1, 1]
total = cm.sum()
corr_pred_per_dt = round(corr_pred/total*100, 2)
print('Percentage of correct predictions: ', corr_pred_per_dt)

# Random Forect Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy')
rf_classifier.fit(X_train, y_train)

y_pred_rf = rf_classifier.predict(X_test)

# Lets measure the accuracy of the model using the confusion matrix.
cm = confusion_matrix(y_test, y_pred_rf)

corr_pred = cm[0, 0] + cm[1, 1]
total = cm.sum()
corr_pred_per_rf = round(corr_pred/total*100, 2)
print('Percentage of correct predictions: ', corr_pred_per_rf)

In [ ]:
# Lets store all the scores in a dataframe
score = pd.DataFrame()
score['Models'] = ('Logistic Regression', 'K Nearest Neighbors', 'Naive Bayes', 'SVM - Linear', 'Kernel SVM', 'Decision Tree', 'Random Forest')
score['Accuracy'] = (corr_pred_per, corr_pred_per_knn, corr_pred_per_nb, corr_pred_per_svc, corr_pred_per_svm, corr_pred_per_dt, corr_pred_per_rf)
score